In [ ]:
import sys
import os
import glob
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import json

# =========================================================
# 0. 路径配置与环境准备
# =========================================================
paths_to_add = ['/root', '/root/task']
for p in paths_to_add:
    if p not in sys.path:
        sys.path.append(p)

# 导入项目模块
from src.YoloSAM.models.sam import SAMModel
from src.YoloSAM.utils.config import SAMFinetuneConfig, SAMDatasetConfig

# 创建评估目录（自动创建子目录：metrics/可视化/报告）
EVALUATE_DIR = "/root/task/evaluate"
os.makedirs(EVALUATE_DIR, exist_ok=True)
os.makedirs(os.path.join(EVALUATE_DIR, "metrics"), exist_ok=True)
os.makedirs(os.path.join(EVALUATE_DIR, "visualizations"), exist_ok=True)
os.makedirs(os.path.join(EVALUATE_DIR, "reports"), exist_ok=True)

# 数据集路径（根据实际情况调整）
VAL_IMAGE_DIR = "/root/task/datasets/DRIVE/val/images"
VAL_LABEL_DIR = "/root/task/datasets/DRIVE/val/label"  # mask目录名：label
IMAGE_EXT = ".tif"
LABEL_EXT = ".gif"  # 标签文件扩展名（根据实际调整）
LABEL_SUFFIX = "manual1"
IMAGE_SIZE = 1024

# =========================================================
# 1. 定义评估指标计算函数
# =========================================================
class SegmentationMetrics:
    """语义分割评估指标计算类"""
    def __init__(self, eps=1e-6):
        self.eps = eps  # 防止除零错误
        self.reset()

    def reset(self):
        """重置所有指标"""
        self.total_tp = 0  # 真阳性
        self.total_fp = 0  # 假阳性
        self.total_fn = 0  # 假阴性
        self.total_tn = 0  # 真阴性
        self.image_metrics = []  # 单张图片的指标

    def calculate_single(self, pred_mask, gt_mask):
        """计算单张图片的指标"""
        # 确保掩码为布尔类型
        pred_mask = pred_mask.astype(bool)
        gt_mask = gt_mask.astype(bool)

        # 计算混淆矩阵元素
        tp = np.logical_and(pred_mask, gt_mask).sum()
        fp = np.logical_and(pred_mask, np.logical_not(gt_mask)).sum()
        fn = np.logical_and(np.logical_not(pred_mask), gt_mask).sum()
        tn = np.logical_and(np.logical_not(pred_mask), np.logical_not(gt_mask)).sum()

        # 计算核心指标
        iou = tp / (tp + fp + fn + self.eps)
        dice = 2 * tp / (2 * tp + fp + fn + self.eps)
        precision = tp / (tp + fp + self.eps)
        recall = tp / (tp + fn + self.eps)
        f1 = 2 * precision * recall / (precision + recall + self.eps)

        return {
            "tp": tp, "fp": fp, "fn": fn, "tn": tn,
            "iou": iou, "dice": dice,
            "precision": precision, "recall": recall, "f1": f1
        }

    def update(self, pred_mask, gt_mask, image_name):
        """更新全局指标"""
        metrics = self.calculate_single(pred_mask, gt_mask)
        self.total_tp += metrics["tp"]
        self.total_fp += metrics["fp"]
        self.total_fn += metrics["fn"]
        self.total_tn += metrics["tn"]
        
        metrics["image_name"] = image_name
        self.image_metrics.append(metrics)

    def get_global_metrics(self):
        """计算全局平均指标"""
        # 全局混淆矩阵（转换为Python原生int）
        tp = int(self.total_tp)
        fp = int(self.total_fp)
        fn = int(self.total_fn)
        tn = int(self.total_tn)
    
        # 全局指标（保留浮点数，JSON支持）
        global_iou = tp / (tp + fp + fn + self.eps)
        global_dice = 2 * tp / (2 * tp + fp + fn + self.eps)
        global_precision = tp / (tp + fp + self.eps)
        global_recall = tp / (tp + fn + self.eps)
        global_f1 = 2 * global_precision * global_recall / (global_precision + global_recall + self.eps)
    
        return {
            "global_iou": float(global_iou),  # 显式转换为float（可选，JSON原生支持）
            "global_dice": float(global_dice),
            "global_precision": float(global_precision),
            "global_recall": float(global_recall),
            "global_f1": float(global_f1),
            "confusion_matrix": {
                "tp": tp, "fp": fp, "fn": fn, "tn": tn  # 已转换为int
            }
        }

# =========================================================
# 2. 改进的推理类（支持评估模式）
# =========================================================
class YoloSAMInference:
    def __init__(self, yolo_path, sam_path, device='cuda'):
        self.device = torch.device(device)
        
        # 加载 YOLO
        print(f"Loading YOLO from: {yolo_path}")
        self.yolo_model = YOLO(yolo_path)
        print("✅ YOLO Model Loaded.")
        
        # 加载微调的 SAMModel
        print(f"Loading Fine-tuned SAM from: {sam_path}")
        self.sam_model = self._load_finetuned_sam(sam_path)
        self.sam_model.to(self.device)
        self.sam_model.eval()
        print("✅ Fine-tuned SAM Model Loaded.")

    def _load_finetuned_sam(self, checkpoint_path):
        config = SAMFinetuneConfig(model_type='vit_b', sam_path='/root/task/checkpoints/sam_vit_b_01ec64.pth')
        model = SAMModel(config)
        
        state_dict = torch.load(checkpoint_path, map_location=self.device)
        if 'model_state_dict' in state_dict:
            print("📦 Checkpoint format detected, extracting 'model_state_dict'...")
            state_dict = state_dict['model_state_dict']
            
        model.load_state_dict(state_dict)
        return model

    def predict(self, image_path, yolo_conf=0.01, yolo_iou=0.5, image_size=1024):
        """推理单张图片，返回预测掩码（合并后）"""
        # 图像预处理
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)
        resized_image_np = cv2.resize(image_np, (image_size, image_size))
        
        # YOLO 推理
        yolo_results = self.yolo_model.predict(resized_image_np, conf=yolo_conf, iou=yolo_iou, verbose=False)
        detected_boxes = []
        if yolo_results and len(yolo_results[0].boxes) > 0:
            detected_boxes = yolo_results[0].boxes.xyxy.cpu()
        
        # SAM 推理
        image_tensor = torch.from_numpy(resized_image_np).permute(2, 0, 1).float() / 255.0
        image_tensor = image_tensor.to(self.device).unsqueeze(0)

        combined_mask = np.zeros((image_size, image_size), dtype=bool)
        if len(detected_boxes) > 0:
            with torch.no_grad():
                for box in detected_boxes:
                    prompt_box = box.unsqueeze(0).to(self.device)
                    pred_mask_logits, _ = self.sam_model.forward_one_image(
                        image=image_tensor,
                        bounding_box=prompt_box,
                        is_train=False
                    )
                    pred_mask_prob = torch.sigmoid(pred_mask_logits)
                    pred_mask_binary = (pred_mask_prob > 0.5).squeeze().cpu().numpy()
                    combined_mask = np.logical_or(combined_mask, pred_mask_binary)

        return {
            "original_image": resized_image_np,
            "detected_boxes": detected_boxes.numpy() if len(detected_boxes) > 0 else [],
            "predicted_mask": combined_mask,  # 合并后的掩码
            "original_shape": image_np.shape[:2]
        }

    def load_gt_mask(self, label_path, image_size=1024):
        """加载并预处理真实标签掩码"""
        gt_mask = Image.open(label_path).convert("L")  # 灰度图
        gt_mask = np.array(gt_mask)
        # 二值化（根据数据集调整阈值，DRIVE通常是0/255）
        gt_mask = (gt_mask > 127).astype(bool)
        # 调整尺寸与预测掩码一致
        gt_mask = cv2.resize(gt_mask.astype(np.uint8), (image_size, image_size)) > 0
        return gt_mask

    def visualize_evaluation(self, image_np, pred_mask, gt_mask, image_name):
        """可视化评估结果（原图+预测掩码+真实掩码+对比）"""
        fig, axes = plt.subplots(2, 2, figsize=(16, 16))
        
        # 1. 原图
        axes[0,0].imshow(image_np)
        axes[0,0].set_title("Original Image")
        axes[0,0].axis('off')
        
        # 2. 预测掩码
        axes[0,1].imshow(pred_mask, cmap='gray')
        axes[0,1].set_title("Predicted Mask")
        axes[0,1].axis('off')
        
        # 3. 真实掩码
        axes[1,0].imshow(gt_mask, cmap='gray')
        axes[1,0].set_title("Ground Truth Mask")
        axes[1,0].axis('off')
        
        # 4. 对比图（TP:绿, FP:红, FN:黄）
        compare_mask = np.zeros((*pred_mask.shape, 3), dtype=np.uint8)
        compare_mask[np.logical_and(pred_mask, gt_mask)] = [0, 255, 0]    # TP: 绿色
        compare_mask[np.logical_and(pred_mask, ~gt_mask)] = [255, 0, 0]   # FP: 红色
        compare_mask[np.logical_and(~pred_mask, gt_mask)] = [255, 255, 0] # FN: 黄色
        axes[1,1].imshow(compare_mask)
        axes[1,1].set_title("Comparison (TP:Green, FP:Red, FN:Yellow)")
        axes[1,1].axis('off')
        
        # 保存可视化图
        save_path = os.path.join(EVALUATE_DIR, "visualizations", f"{image_name}_eval.png")
        plt.tight_layout()
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()

# =========================================================
# 3. 主评估流程
# =========================================================
def main_evaluation():
    # 1. 加载权重
    # 找 YOLO
    yolo_files = glob.glob("/root/task/src/YoloSAM/drive/runs/**/weights/best.pt", recursive=True)
    if not yolo_files: raise FileNotFoundError("找不到任何 YOLO 的 best.pt！")
    latest_yolo_path = "/root/task/src/YoloSAM/drive/runs/yolo_vessel_detection3/weights/best.pt"
    print(f"👉 Found latest YOLO weights: {latest_yolo_path}")

    # 找 SAM
    sam_files = glob.glob("/root/autodl-tmp/run_*/best_model.pth", recursive=True)
    if not sam_files: raise FileNotFoundError("找不到任何微调的 SAM best_model.pth！")
    latest_sam_path = max(sam_files, key=os.path.getmtime)
    print(f"👉 Found latest SAM weights: {latest_sam_path}")

    # 2. 初始化推理器和指标计算器
    pipeline = YoloSAMInference(
        yolo_path=latest_yolo_path,
        sam_path=latest_sam_path,
        device='cuda'
    )
    metrics_calculator = SegmentationMetrics()

    # 3. 获取验证集图片列表
    val_image_paths = glob.glob(os.path.join(VAL_IMAGE_DIR, f"*{IMAGE_EXT}"))
    if not val_image_paths:
        raise FileNotFoundError(f"在 {VAL_IMAGE_DIR} 中未找到 {IMAGE_EXT} 格式的图片！")
    print(f"\n📊 开始评估，共 {len(val_image_paths)} 张验证图片")

    # 4. 批量推理+评估
    for image_path in tqdm(val_image_paths, desc="Evaluating"):
        # 获取图片名称（用于保存）
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        
        # 对应的标签路径
        label_path = os.path.join(VAL_LABEL_DIR, f"{image_name}_{LABEL_SUFFIX}{LABEL_EXT}")
        label_path = label_path.replace('_test', '')
        if not os.path.exists(label_path):
            print(f"⚠️  跳过 {image_name}：未找到标签文件 {label_path}")
            continue

        # 推理
        results = pipeline.predict(image_path, image_size=IMAGE_SIZE)
        pred_mask = results["predicted_mask"]
        
        # 加载真实掩码
        gt_mask = pipeline.load_gt_mask(label_path, image_size=IMAGE_SIZE)
        
        # 计算并更新指标
        metrics_calculator.update(pred_mask, gt_mask, image_name)
        
        # 可视化并保存
        pipeline.visualize_evaluation(results["original_image"], pred_mask, gt_mask, image_name)

    # 5. 计算全局指标
    global_metrics = metrics_calculator.get_global_metrics()
    print("\n==================== 全局评估指标 ====================")
    print(f"全局 IoU: {global_metrics['global_iou']:.4f}")
    print(f"全局 Dice: {global_metrics['global_dice']:.4f}")
    print(f"全局 Precision: {global_metrics['global_precision']:.4f}")
    print(f"全局 Recall: {global_metrics['global_recall']:.4f}")
    print(f"全局 F1-Score: {global_metrics['global_f1']:.4f}")
    print("======================================================")

    # 6. 保存评估结果
    # 6.1 保存单张图片指标（CSV）
    single_metrics_df = pd.DataFrame(metrics_calculator.image_metrics)
    single_metrics_path = os.path.join(EVALUATE_DIR, "metrics", "single_image_metrics.csv")
    single_metrics_df.to_csv(single_metrics_path, index=False, encoding="utf-8")

    # 6.2 保存全局指标（JSON）
    global_metrics_path = os.path.join(EVALUATE_DIR, "reports", "global_metrics.json")
    with open(global_metrics_path, "w", encoding="utf-8") as f:
        json.dump(global_metrics, f, indent=4, ensure_ascii=False)

    # 6.3 生成混淆矩阵可视化
    cm = np.array([
        [global_metrics['confusion_matrix']['tn'], global_metrics['confusion_matrix']['fp']],
        [global_metrics['confusion_matrix']['fn'], global_metrics['confusion_matrix']['tp']]
    ])
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Negative', 'Positive'],
                yticklabels=['Negative', 'Positive'])
    plt.xlabel('Predicted')
    plt.ylabel('Ground Truth')
    plt.title('Confusion Matrix (Global)')
    cm_save_path = os.path.join(EVALUATE_DIR, "reports", "confusion_matrix.png")
    plt.savefig(cm_save_path, dpi=150, bbox_inches='tight')
    plt.close()

    # 6.4 生成指标汇总报告（TXT）
    report_path = os.path.join(EVALUATE_DIR, "reports", "evaluation_report.txt")
    with open(report_path, "w", encoding="utf-8") as f:
        f.write("==================== YoloSAM 模型评估报告 ====================\n")
        f.write(f"评估时间: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"验证集数量: {len(val_image_paths)}\n")
        f.write(f"YOLO 权重路径: {latest_yolo_path}\n")
        f.write(f"SAM 权重路径: {latest_sam_path}\n")
        f.write("\n【全局指标】\n")
        f.write(f"IoU: {global_metrics['global_iou']:.4f}\n")
        f.write(f"Dice 系数: {global_metrics['global_dice']:.4f}\n")
        f.write(f"精确率 (Precision): {global_metrics['global_precision']:.4f}\n")
        f.write(f"召回率 (Recall): {global_metrics['global_recall']:.4f}\n")
        f.write(f"F1 分数: {global_metrics['global_f1']:.4f}\n")
        f.write("\n【混淆矩阵】\n")
        f.write(f"真阴性 (TN): {global_metrics['confusion_matrix']['tn']}\n")
        f.write(f"假阳性 (FP): {global_metrics['confusion_matrix']['fp']}\n")
        f.write(f"假阴性 (FN): {global_metrics['confusion_matrix']['fn']}\n")
        f.write(f"真阳性 (TP): {global_metrics['confusion_matrix']['tp']}\n")
        f.write("==============================================================\n")

    print(f"\n✅ 评估完成！所有结果已保存到：{EVALUATE_DIR}")
    print(f"   - 单张图片指标: {single_metrics_path}")
    print(f"   - 全局指标: {global_metrics_path}")
    print(f"   - 评估报告: {report_path}")
    print(f"   - 混淆矩阵: {cm_save_path}")
    print(f"   - 可视化结果: {os.path.join(EVALUATE_DIR, 'visualizations')}")

# =========================================================
# 执行评估
# =========================================================
if __name__ == "__main__":
    main_evaluation()

👉 Found latest YOLO weights: /root/task/src/YoloSAM/drive/runs/yolo_vessel_detection3/weights/best.pt
👉 Found latest SAM weights: /root/autodl-tmp/run_25/best_model.pth
Loading YOLO from: /root/task/src/YoloSAM/drive/runs/yolo_vessel_detection3/weights/best.pt
✅ YOLO Model Loaded.
Loading Fine-tuned SAM from: /root/autodl-tmp/run_25/best_model.pth
Load SAM model from  /root/task/checkpoints/sam_vit_b_01ec64.pth
📦 Checkpoint format detected, extracting 'model_state_dict'...
✅ Fine-tuned SAM Model Loaded.

📊 开始评估，共 20 张验证图片


Evaluating: 100%|██████████| 20/20 [00:45<00:00,  2.24s/it]